In [1]:
#Importing required Datasets
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split  

In [2]:
data = pd.read_csv("D:\Work\Imarticus\Machine Learning\Exam 2\\Sentiment.csv")

In [3]:
#Previewing the Dataset
data.head()

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


In [4]:
data.shape #We have 13,871 observations along with 21 columns

(13871, 21)

In [5]:
#We need only 'text' and 'sentiments' columns for this dataset.
new_data = data[['text','sentiment']]
print(new_data)

                                                    text sentiment
0      RT @NancyLeeGrahn: How did everyone feel about...   Neutral
1      RT @ScottWalker: Didn't catch the full #GOPdeb...  Positive
2      RT @TJMShow: No mention of Tamir Rice and the ...   Neutral
3      RT @RobGeorge: That Carly Fiorina is trending ...  Positive
4      RT @DanScavino: #GOPDebate w/ @realDonaldTrump...  Positive
...                                                  ...       ...
13866  RT @cappy_yarbrough: Love to see men who will ...  Negative
13867  RT @georgehenryw: Who thought Huckabee exceede...  Positive
13868  RT @Lrihendry: #TedCruz As President, I will a...  Positive
13869  RT @JRehling: #GOPDebate Donald Trump says tha...  Negative
13870  RT @Lrihendry: #TedCruz headed into the Presid...  Positive

[13871 rows x 2 columns]


In [6]:
#We are checking the total number of 'positive' , 'negative' sentiments we have.
new_data['sentiment'].value_counts()

Negative    8493
Neutral     3142
Positive    2236
Name: sentiment, dtype: int64

In [7]:
#We need to classify tweets as either negative or positive, so we will filter out rows with neutral sentiment.

new_data = new_data[new_data['sentiment'] != 'Neutral']

In [8]:
new_data.head()

,text,sentiment
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
5,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",Positive
6,RT @warriorwoman91: I liked her and was happy ...,Negative


In [9]:
#convert sentiment to numeric
sentiment_label = new_data.sentiment.factorize()

sentiment_label

(array([0, 0, 0, ..., 0, 1, 0], dtype=int64),
 Index(['Positive', 'Negative'], dtype='object'))

In [10]:
pos = new_data[new_data['sentiment'] == 'Positive']
pos = pos['text']
neg = new_data[new_data['sentiment'] == 'Negative']
neg = neg['text']

In [11]:
#We are now building a LSTM model to predict the sentiments.

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tweet = new_data.text.values
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(tweet)
vocab_size = len(tokenizer.word_index) + 1
encoded_docs = tokenizer.texts_to_sequences(tweet)
padded_sequence = pad_sequences(encoded_docs, maxlen=200)

We get actual texts from the dataframe.
We are limiting the number of words to 5000.

In [12]:
#index is assigned to all the words present in the dataframe.
print(tokenizer.word_index)

{'the': 1, 'gopdebate': 2, 'rt': 3, 'gopdebates': 4, 'to': 5, 'is': 6, 'of': 7, 'a': 8, 'and': 9, 't': 10, 'co': 11, 'i': 12, 'http': 13, 'rwsurfergirl': 14, '🇺🇸': 15, 'you': 16, 'it': 17, 'in': 18, 'for': 19, 'trump': 20, 'that': 21, 'fox': 22, 'on': 23, 'not': 24, 'this': 25, 'was': 26, 'realdonaldtrump': 27, 'about': 28, 'amp': 29, 'debate': 30, 'news': 31, 'from': 32, 'have': 33, 'at': 34, 'we': 35, 'but': 36, 'he': 37, 'me': 38, 'be': 39, 'just': 40, 'candidates': 41, 'are': 42, '…': 43, 'like': 44, 'last': 45, 'so': 46, 'they': 47, 'up': 48, 'megynkelly': 49, 'https': 50, 'all': 51, 'with': 52, 'gop': 53, "don't": 54, 'what': 55, 'people': 56, 'jeb': 57, 'who': 58, 'foxnews': 59, 'my': 60, 'how': 61, 'as': 62, 'bush': 63, 'if': 64, 'night': 65, 'no': 66, 'has': 67, 'their': 68, 'by': 69, 'when': 70, 'can': 71, 'do': 72, 'more': 73, 'out': 74, 'one': 75, 'should': 76, 'or': 77, "i'm": 78, 'his': 79, 'would': 80, 'chris': 81, 'him': 82, 'get': 83, 'think': 84, 'republican': 85, 'pr

In [13]:
print(tweet[1])
print(encoded_docs[1])

RT @RobGeorge: That Carly Fiorina is trending -- hours after HER debate -- above any of the men in just-completed #GOPdebate says she's on …
[3, 21, 320, 259, 6, 4369, 843, 107, 173, 30, 2845, 139, 7, 1, 176, 18, 40, 2, 236, 738, 23, 43]


In [14]:
print(padded_sequence[1])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    3   21  320  259
    6 4369  843  107  173   30 2845  139    7    1  176   18   40    2
  236 

In [15]:
#Building the Model with LSTM

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM , Dense, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import Embedding
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,     
                                     input_length=200) )
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 32)           506240    
                                                                 
 spatial_dropout1d (SpatialD  (None, 200, 32)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 50)                16600     
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 522,891
Trainable params: 522,891
Non-trainable params: 0
__________________________________________________

In [16]:
#Train the model

history = model.fit(padded_sequence,sentiment_label[0],
                  validation_split=0.2, epochs=50, batch_size=32)

Epoch 1/50
269/269 [==============================] - 81s 274ms/step - loss: 0.4775 - accuracy: 0.8010 - val_loss: 0.4055 - val_accuracy: 0.8262
Epoch 2/50
269/269 [==============================] - 71s 264ms/step - loss: 0.3378 - accuracy: 0.8617 - val_loss: 0.3256 - val_accuracy: 0.8756
Epoch 3/50
269/269 [==============================] - 70s 260ms/step - loss: 0.2824 - accuracy: 0.8848 - val_loss: 0.3412 - val_accuracy: 0.8658
Epoch 4/50
269/269 [==============================] - 69s 256ms/step - loss: 0.2444 - accuracy: 0.9006 - val_loss: 0.3323 - val_accuracy: 0.8667
Epoch 5/50
269/269 [==============================] - 68s 253ms/step - loss: 0.2200 - accuracy: 0.9097 - val_loss: 0.3618 - val_accuracy: 0.8611
Epoch 6/50
269/269 [==============================] - 69s 255ms/step - loss: 0.2047 - accuracy: 0.9184 - val_loss: 0.3709 - val_accuracy: 0.8625
Epoch 7/50
269/269 [==============================] - 72s 269ms/step - loss: 0.1819 - accuracy: 0.9261 - val_loss: 0.4029 - val_ac

We have an accuracy of 97% for our model

In [ ]:
#Based on the model we have built we are going to check the sentiment analysis of few sentences.

In [17]:
#Statement 1: 'He is a great leader.'

test_word1 ="He is a great leader"
tw = tokenizer.texts_to_sequences([test_word1])
tw = pad_sequences(tw,maxlen=200)
prediction = int(model.predict(tw).round().item())
sentiment_label[1][prediction]

'Positive'

In [24]:
#Statement 2: 'He is a terrible politician.'

test_word2 ="He is a terrible politician"
tw = tokenizer.texts_to_sequences([test_word2])
tw = pad_sequences(tw,maxlen=200)
prediction = int(model.predict(tw).round().item())
sentiment_label[1][prediction]

'Negative'

In [28]:
#Statement 3: 'I hate your taste.'

test_word4 ="I hate you taste"
tw = tokenizer.texts_to_sequences([test_word4])
tw = pad_sequences(tw,maxlen=200)
prediction = int(model.predict(tw).round().item())
sentiment_label[1][prediction]

'Negative'

We have successfully built a model which can predict the sentiment of a statement with an accuracy of 97%. Now we could have further made it more effective by using the Neutral Sentiment as well but for this we have only worked with 'Positive' and 'Negative' sentiments.